In [1]:
!pip install gradio opencv-python mediapipe numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found exist

In [1]:
import gradio as gr
import cv2
import numpy as np
import mediapipe as mp
from datetime import datetime

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

def enhance_low_light(image, brightness_factor=1.5):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    v = clahe.apply(v)
    v = cv2.convertScaleAbs(v, alpha=brightness_factor, beta=10)
    hsv = cv2.merge([h, s, v])
    enhanced = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return enhanced

def detect_faces(image):
    results = face_detection.process(image)
    height, width, _ = image.shape
    face_locations = []
    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            x = int(bbox.xmin * width)
            y = int(bbox.ymin * height)
            w = int(bbox.width * width)
            h = int(bbox.height * height)
            x = max(0, x)
            y = max(0, y)
            w = min(w, width - x)
            h = min(h, height - y)
            face_locations.append((x, y, w, h))
    return face_locations

def anonymize_faces(image, face_locations):
    anonymized = image.copy()
    for (x, y, w, h) in face_locations:
        face_roi = anonymized[y:y+h, x:x+w]
        if face_roi.size > 0:
            small = cv2.resize(face_roi, (15, 15), interpolation=cv2.INTER_LINEAR)
            pixelated = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)
            anonymized[y:y+h, x:x+w] = pixelated
    return anonymized

def process(image, low_light, anonymize):
    if image is None:
        return None, "Please upload an image."
    # image is already a numpy array (H, W, C) in RGB
    if low_light:
        image = enhance_low_light(image)
    face_locations = detect_faces(image)
    count = len(face_locations)
    for (x, y, w, h) in face_locations:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0), 2)
    if anonymize:
        image = anonymize_faces(image, face_locations)
    cv2.putText(image, f"Students: {count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log = f"[{timestamp}] Students present: {count}"
    return image, log


with gr.Blocks() as demo:
    gr.Markdown("# 🏫 Smart Classroom Face Detection System")
    gr.Markdown("Upload an image to detect faces and count students.")
    with gr.Row():
        with gr.Column():
            img_upload = gr.Image(label="Upload Image")
            low_light = gr.Checkbox(label="Enable Low-Light Enhancement", value=False)
            anonymize = gr.Checkbox(label="Anonymize Faces (Pixelate)", value=True)
            btn = gr.Button("Detect Faces")
        with gr.Column():
            out_img = gr.Image(label="Result", interactive=False)
            log = gr.Textbox(label="Attendance Log", interactive=False)
    btn.click(process, inputs=[img_upload, low_light, anonymize], outputs=[out_img, log])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://679909b110bd81f9ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
